In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import cv2
import re

mpl.rcParams['font.sans-serif'] = ['SimHei']
%matplotlib inline

for lib in [('numpy',np), ('pandas', pd), ('matplotlib', mpl), ('opencv', cv2)]:
    print(f'{lib[0]:>10} version is {lib[1].__version__}')

     numpy version is 1.14.3
    pandas version is 0.23.0
matplotlib version is 2.2.2
    opencv version is 4.1.1


In [6]:
def test_train_file_names(k, d):
    """
    将指定目录下的文件分组，每组随机提取 k 比率的文件作为训练集，其余的作为测试集。
    注意：该函数只对文件名操作，并非实际对文件操作
    
    Paramaters
    -----------
    k -- 训练集所占比率 0<k<=1
    d -- 存放文件的目录
    
    Returns
    -------
    ndarray 1-D 训练集
    ndarray 1-D 测试集
    """
    # 构建的DataFrame的字段名称
    FILE_NAME = 'file_name'
    FILE_GROUP_NAME = 'file_group_name'
    # 获取文件列表
    fn_list = os.listdir(d)
    # 根据文件列表创建 DataFrame
    df = pd.DataFrame(fn_list, columns=[FILE_NAME])
    # 增加文件文组标示
    # extract 的说明: 根据正则表达式 r'(.*)(_)' 提取字符串
    #                比如: 's1_1.bmp' -> ('s1', '_')
    #                     's10_10.bmp' -> ('s10', '_')
    df[FILE_GROUP_NAME] = df.file_name.str.extract(r'(.*)(_)')[0]
    # 根据 FILE_GROUP_NAME 进行分组, 得到分组对象 grouped
    # 分组对象 grouded 的 groups 属性是一个字典, key 是分组名称, value 是分组数据(row index)
    # 形如: {'s1': [1,2,3,...], 's2': [10,11,12...]}
    grouped = df.groupby([FILE_GROUP_NAME])
    train_set = np.array([])
    test_set = np.array([])
    # 循环所有分组数据, 每次操作一组数据
    for v in grouped.groups.values():
        # 从 DF 中获取 file_name 字段 的数据
        # v 是 row index.
        data = df[FILE_NAME][v]
        # 随机获取指定比率的数据 作为训练集(文件名称)
        train = data.sample(frac=k)
        # 提取剩余数据 作为测试集(文件名称)
        test = data.drop(train.index)
        # 将一组中的随机提取的训练集数据放入总训练集中
        train_set = np.concatenate((train_set, train.values))
        # 将一组中的测试集数据放入总测试集中
        test_set = np.concatenate((test_set, test.values))
    return train_set, test_set

def img2vector(img_file_path_name):
    """
    图片向量化
    将给定的图片(m, n), 输出(1, m*n)的行向量
    """
    img = cv2.imread(img_file_path_name, 0)
    return img.flatten().reshape(1, -1)

def load_orl(k, d):
    """
    加载图片文件形成 matrix
    
    Parameters
    ----------
    k {float} -- 图片文件训练集比率 0 < k <= 1
    d {string} -- 图片文件所在目录
    
    Returns
    -------
    {ndarray 2D} -- 训练集(特征)
    {ndarray 2D} -- 训练集(标签/真实值)
    {ndarray 2D} -- 测试集(特征)
    {ndarray 2D} -- 测试集(标签/真实值)
    """
    # 图片文件拆分为 训练集用文件 测试集用文件
    train_file_names, test_file_names = test_train_file_names(k, d)
    # 生成测试集第一条数据
    train_set = img2vector(os.path.join(d, train_file_names[0]))
    # 生成测试集第一个标签
    train_label = np.array([re.sub('_.*', '', train_file_names[0])])
    # 生成训练集第一条数据
    test_set = img2vector(os.path.join(d, test_file_names[0]))
    # 生成训练集第一个标签
    test_label = np.array([re.sub('_.*', '', test_file_names[0])])
    # 循环生成训练集数据和训练集标签
    # 因为第一条数据已经生成过, 因此需要从第二条开始循环
    for f in train_file_names[1:]:
        img = img2vector(os.path.join(d, f))
        train_set = np.concatenate((train_set, img))
        train_label = np.append(train_label, re.sub('_.*', '', f))
    # 循环生成测试集数据和训练集标签
    # 因为第一条数据已经生成过, 因此需要从第二条开始循环
    for f in test_file_names[1:]:
        img = img2vector(os.path.join(d, f))
        test_set = np.concatenate((test_set, img))
        test_label = np.append(test_label, re.sub('_.*', '', f))
    return train_set, train_label, test_set, test_label

def PCA(data, r):
    data_ = np.copy(data)
    mu = data_.mean(axis=0)
    A = data_ - mu
    C = A @ A.T
    

In [60]:
x = np.random.randn(5,3)
m = x.mean(axis=0)
A = (x - m).T
C = (A @ A.T) / x.shape[0]
print('协方差矩阵:')
print(C)
D, V = np.linalg.eig(C)
print(f'特征值: {D}')
print('特征向量:')
print(V)
V @ C @ V.T

协方差矩阵:
[[ 0.17851398 -0.07226306 -0.23296255]
 [-0.07226306  0.1753461   0.28375326]
 [-0.23296255  0.28375326  0.88723054]]
特征值: [1.05434018 0.11260123 0.07414921]
特征向量:
[[ 0.26814727 -0.93875331  0.21642379]
 [-0.31584907 -0.29790052 -0.90082997]
 [-0.91012988 -0.17319784  0.37638557]]


array([[ 0.10296001,  0.1391497 , -0.08968878],
       [ 0.1391497 ,  0.75948065, -0.4148551 ],
       [-0.08968878, -0.4148551 ,  0.37864996]])